Preprocessing

In [8]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

DATASET_DIR = "D:/SFT_ATTEMPT2/raw_video"  # <-- adjust full path to your dataset
TARGET_WORDS = ["Halo", "Kamu", "Apa", "Dimana", "Duduk"]
IMG_SIZE = 96
FRAMES_PER_VIDEO = 20  # sample fixed length from each video

def load_videos():
    X, y = [], []
    for word in TARGET_WORDS:
        word_path = os.path.join(DATASET_DIR, word)
        for vid_file in os.listdir(word_path):
            vid_path = os.path.join(word_path, vid_file)
            cap = cv2.VideoCapture(vid_path)

            frames = []
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = frame.astype("float32") / 255.0
                frames.append(frame)

            cap.release()

            # Uniformly sample FRAMES_PER_VIDEO frames
            if len(frames) >= FRAMES_PER_VIDEO:
                idxs = np.linspace(0, len(frames)-1, FRAMES_PER_VIDEO, dtype=int)
                frames = [frames[i] for i in idxs]
                X.append(frames)
                y.append(TARGET_WORDS.index(word))

    return np.array(X), np.array(y)

X, y = load_videos()
print("Shape:", X.shape, y.shape)  # (num_samples, FRAMES_PER_VIDEO, 96, 96, 3)

# Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


Shape: (250, 20, 96, 96, 3) (250,)


In [15]:
# preprocess.py
import os, glob, json
import numpy as np
import cv2
import mediapipe as mp
from tqdm import tqdm

DATASET_DIR = "D:/SFT_ATTEMPT2/raw_video"
TARGET_WORDS = ["Halo","Kamu","Apa","Dimana","Duduk"]
CACHE_DIR = "cache_landmarks"
SEQ_LEN = 20
TARGET_FPS = 24

os.makedirs(CACHE_DIR, exist_ok=True)

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, model_complexity=0,
                       max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)

def normalize_hand(landmarks, w, h):
    """Return 63-d vector for 21 landmarks (x,y,z) normalized relative to wrist & scale.
       If landmarks is None return zeros."""
    if landmarks is None:
        return np.zeros(63, dtype=np.float32)
    pts = np.array([(lm.x, lm.y, lm.z) for lm in landmarks], dtype=np.float32)
    # center at wrist (index 0)
    center = pts[0].copy()
    pts[:, :2] -= center[:2]
    px = pts[:,0]*w
    py = pts[:,1]*h
    scale = max(px.max()-px.min(), py.max()-py.min(), 1e-3)
    pts[:, :2] /= (scale / max(w,h))
    return pts.flatten().astype(np.float32)

def extract_sequences_from_video(path, seq_len=SEQ_LEN, target_fps=TARGET_FPS):
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        return []
    src_fps = cap.get(cv2.CAP_PROP_FPS) or 30
    step = max(int(round(src_fps / target_fps)), 1)

    frames_feats = []
    i = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if i % step != 0:
            i += 1
            continue
        h,w = frame.shape[:2]
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = hands.process(rgb)

        left = None; right = None
        if res.multi_hand_landmarks and res.multi_handedness:
            # pair them
            for lm, handed in zip(res.multi_hand_landmarks, res.multi_handedness):
                if handed.classification[0].label.lower() == "left":
                    left = lm.landmark
                else:
                    right = lm.landmark

        L = normalize_hand(left, w, h)
        R = normalize_hand(right, w, h)
        present = np.array([1.0 if left is not None else 0.0, 1.0 if right is not None else 0.0], dtype=np.float32)
        feat = np.concatenate([L, R, present])    # length = 63+63+2 = 128
        frames_feats.append(feat)
        i += 1

    cap.release()
    if len(frames_feats) < seq_len:
        return []

    seqs = []
    for start in range(0, len(frames_feats) - seq_len + 1, seq_len):
        seqs.append(np.stack(frames_feats[start:start+seq_len], axis=0))
    return seqs

# Scan dataset
X=[]; y=[]
label_map = {w:i for i,w in enumerate(TARGET_WORDS)}
for w in TARGET_WORDS:
    folder = os.path.join(DATASET_DIR, w)
    if not os.path.isdir(folder): 
        print("Missing folder:", folder); continue
    vids = glob.glob(os.path.join(folder, "*.mp4"))
    print(f"Found {len(vids)} videos for {w}")
    for v in tqdm(vids):
        seqs = extract_sequences_from_video(v)
        for s in seqs:
            X.append(s)   # shape (T,128)
            y.append(label_map[w])

X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.int64)

print("Built dataset shapes:", X.shape, y.shape)
np.save(os.path.join(CACHE_DIR, "X.npy"), X)
np.save(os.path.join(CACHE_DIR, "y.npy"), y)
with open(os.path.join(CACHE_DIR, "labels.json"), "w") as f:
    json.dump(TARGET_WORDS, f)
print("Saved to", CACHE_DIR)


Found 50 videos for Halo


100%|██████████| 50/50 [00:55<00:00,  1.11s/it]


Found 50 videos for Kamu


100%|██████████| 50/50 [00:39<00:00,  1.27it/s]


Found 50 videos for Apa


...

Found 50 videos for Dimana


100%|██████████| 50/50 [00:45<00:00,  1.10it/s]


Found 50 videos for Duduk


100%|██████████| 50/50 [01:20<00:00,  1.62s/it]

Built dataset shapes: (773, 20, 128) (773,)
Saved to cache_landmarks


In [17]:
# train.py
import os
import math
import json
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# -------------------------
# Config
# -------------------------
CACHE_DIR = "cache_landmarks"
SEQ_LEN = 20
MODEL_PATH = "bisindo_landmarks.keras"   # recommended Keras v3 format
BATCH_SIZE = 32
EPOCHS = 80

# -------------------------
# Load data
# -------------------------
X = np.load(os.path.join(CACHE_DIR, "X.npy"))  # expected shape (N, T, F)
y = np.load(os.path.join(CACHE_DIR, "y.npy"))
with open(os.path.join(CACHE_DIR, "labels.json"), "r", encoding="utf-8") as f:
    labels = json.load(f)

print("Loaded data shapes:", X.shape, y.shape)
num_classes = len(labels)

# -------------------------
# Train / val split
# -------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print("Train:", X_train.shape, y_train.shape, "Val:", X_val.shape, y_val.shape)

# -------------------------
# Augmentation helper
# -------------------------
def augment_batch(X_batch, prob=0.5):
    """Add small gaussian jitter to landmarks with probability per sample."""
    Xb = X_batch.copy()
    for i in range(len(Xb)):
        if np.random.rand() < prob:
            noise = np.random.normal(0, 0.01, Xb[i].shape).astype(np.float32)
            Xb[i] = Xb[i] + noise
    return Xb

# -------------------------
# Compute class weights and map to sample weights
# -------------------------
class_weights_arr = compute_class_weight("balanced", classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights_arr))
print("Class weights:", class_weights)

# -------------------------
# Generator that yields (Xb, yb, sample_weight)
# -------------------------
def gen_with_weights(Xa, ya, batch_size=BATCH_SIZE):
    n = len(Xa)
    idx = np.arange(n)
    while True:
        np.random.shuffle(idx)
        for i in range(0, n, batch_size):
            b = idx[i:i+batch_size]
            Xb = Xa[b].copy()
            Xb = augment_batch(Xb, prob=0.5)
            yb = ya[b]
            # sample weight per item based on its class
            sw = np.array([class_weights[int(lbl)] for lbl in yb], dtype=np.float32)
            yield Xb, yb, sw

# -------------------------
# Build model (LSTM on landmarks)
# -------------------------
input_shape = X_train.shape[1:]  # (T, F)
inputs = keras.Input(shape=input_shape)
x = keras.layers.Masking(mask_value=0.0)(inputs)
x = keras.layers.LSTM(128, return_sequences=True)(x)
x = keras.layers.Dropout(0.25)(x)
x = keras.layers.LSTM(64)(x)
x = keras.layers.Dropout(0.25)(x)
x = keras.layers.Dense(64, activation="relu")(x)
outputs = keras.layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

# -------------------------
# Callbacks
# -------------------------
cb = [
    keras.callbacks.ModelCheckpoint(MODEL_PATH, save_best_only=True, monitor="val_accuracy", mode="max", verbose=1),
    keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5, verbose=1),
    keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True, verbose=1)
]

# -------------------------
# Fit using generator that yields sample weights
# -------------------------
steps_per_epoch = max(1, math.ceil(len(X_train) / BATCH_SIZE))
print("Steps per epoch:", steps_per_epoch)

history = model.fit(
    gen_with_weights(X_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=steps_per_epoch,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    callbacks=cb,
    verbose=2
)

# ensure final save in Keras format
model.save(MODEL_PATH)
print("Training finished. Model saved to", MODEL_PATH)


Loaded data shapes: (773, 20, 128) (773,)
Train: (618, 20, 128) (618,) Val: (155, 20, 128) (155,)
Class weights: {0: 0.8524137931034482, 1: 1.0747826086956522, 2: 1.0564102564102564, 3: 1.03, 4: 1.0214876033057851}


Model: "functional_81"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 20, 128)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 20, 128)   │          0 │ input_layer_1[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_1 (Masking) │ (None, 20, 128)   │          0 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any_1 (Any)         │ (None, 20)        │          0 │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 20, 128)   │    131,584 │ masking_1[0][0],  │
│                     │                   │            │ any_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 20, 128)   │          0 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 64)        │     49,408 │ dropout_2[0][0],  │
│                     │                   │            │ any_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 64)        │          0 │ lstm_3[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      4,160 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 5)         │        325 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 185,477 (724.52 KB)

 Trainable params: 185,477 (724.52 KB)

 Non-trainable params: 0 (0.00 B)

Steps per epoch: 20
Epoch 1/80

Epoch 1: val_accuracy improved from -inf to 0.81290, saving model to bisindo_landmarks.keras
20/20 - 5s - 254ms/step - accuracy: 0.6440 - loss: 1.1415 - val_accuracy: 0.8129 - val_loss: 0.6547 - learning_rate: 0.0010
Epoch 2/80

Epoch 2: val_accuracy improved from 0.81290 to 0.87097, saving model to bisindo_landmarks.keras
20/20 - 0s - 22ms/step - accuracy: 0.8738 - loss: 0.4228 - val_accuracy: 0.8710 - val_loss: 0.3642 - learning_rate: 0.0010
Epoch 3/80

Epoch 3: val_accuracy improved from 0.87097 to 0.89032, saving model to bisindo_landmarks.keras
20/20 - 0s - 21ms/step - accuracy: 0.9207 - loss: 0.2708 - val_accuracy: 0.8903 - val_loss: 0.3343 - learning_rate: 0.0010
Epoch 4/80

Epoch 4: val_accuracy improved from 0.89032 to 0.89677, saving model to bisindo_landmarks.keras
20/20 - 0s - 23ms/step - accuracy: 0.9482 - loss: 0.1781 - val_accuracy: 0.8968 - val_loss: 0.2977 - learning_rate: 0.0010
Epoch 5/80

Epoch 5: val_accuracy improved from 0.89677 to

In [18]:
import cv2, numpy as np, json, tensorflow as tf
from pathlib import Path
# assumes you have the normalize function from preprocess.py (or reuse code)

# quick helper: extract first seq of length SEQ_LEN from a video using mediapipe landmarks
def extract_seq_from_video(video_path, seq_len=20, target_fps=24):
    import mediapipe as mp
    cap = cv2.VideoCapture(video_path)
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2,
                           min_detection_confidence=0.5, min_tracking_confidence=0.5)
    feats=[]
    i=0
    src_fps = cap.get(cv2.CAP_PROP_FPS) or 30
    step = max(int(round(src_fps/target_fps)),1)
    while True:
        ret, frame = cap.read()
        if not ret: break
        if i % step != 0:
            i+=1; continue
        h,w = frame.shape[:2]
        res = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        left=None; right=None
        if res.multi_hand_landmarks and res.multi_handedness:
            for lm, handed in zip(res.multi_hand_landmarks, res.multi_handedness):
                if handed.classification[0].label.lower()=="left": left=lm.landmark
                else: right=lm.landmark
        # use normalize_hand function from preprocess.py (copy it here)
        def normalize_hand(landmarks,w,h):
            if landmarks is None: return np.zeros(63,dtype=np.float32)
            pts=np.array([(lm.x,lm.y,lm.z) for lm in landmarks],dtype=np.float32)
            center=pts[0].copy(); pts[:,:2]-=center[:2]
            px=pts[:,0]*w; py=pts[:,1]*h
            scale=max(px.max()-px.min(), py.max()-py.min(), 1e-3)
            pts[:,:2]/=(scale/max(w,h))
            return pts.flatten().astype(np.float32)
        L=normalize_hand(left,w,h); R=normalize_hand(right,w,h)
        pres=np.array([1.0 if left else 0.0, 1.0 if right else 0.0], dtype=np.float32)
        feats.append(np.concatenate([L,R,pres]))
        i+=1
        if len(feats)>=seq_len: break
    cap.release()
    hands.close()
    if len(feats)<seq_len: return None
    return np.stack(feats[:seq_len], axis=0)

model = tf.keras.models.load_model("bisindo_landmarks.keras")
labels = json.load(open("cache_landmarks/labels.json"))

seq = extract_seq_from_video("D:/SFT_ATTEMPT2/raw_video/Duduk/Duduk_001.mp4", seq_len=20)
if seq is None:
    print("Not enough frames or missing landmarks")
else:
    pred = model.predict(np.expand_dims(seq,0))[0]
    print("Top probs:", sorted([(labels[i], float(pred[i])) for i in range(len(pred))], key=lambda x:-x[1])[:5])


Not enough frames or missing landmarks
